# Worksheet 08

Name: Jian Xie  
UID: U75516303 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.520545555454363, 4.907864688284163, 4.002100324628296, 5.203945958354401, 5.1721341473236935, 4.815602654878374, 4.0704126327925145, 4.829953004896432, 4.4015884453799305, 6.99753873643184]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.386269997787718, 7.187676010654952, 8.028995364656618, 7.182784194363379, 7.357399382094217, 8.849337990008982, 7.643669665081308, 7.388997521364456, 7.51647897875328, 9.594807152750198]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[9.594807152750198, 6.99753873643184, 7.51647897875328, 4.4015884453799305, 4.829953004896432, 7.388997521364456, 4.0704126327925145, 7.643669665081308, 4.815602654878374, 8.849337990008982]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. The means of the two components: 5 and 8
2. The variances of the two components: 1 and 1
3. The proportion of points in each cluster: 0.5 and 0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.4015884453799305, 4.829953004896432, 4.0704126327925145, 4.815602654878374]
[9.594807152750198, 6.99753873643184, 7.51647897875328, 7.388997521364456, 7.643669665081308, 8.849337990008982]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.5293891844868135,  mean_2 = 7.998471674065011
var_1 = 0.09981231617356531,  var_2 = 0.8339645600798159


They are close to the true values except var_1.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  9.594807152750198
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.0765842099967542
point =  6.99753873643184
probability of observing that point if it came from cluster 0 =  6.650419407009963e-133
probability of observing that point if it came from cluster 1 =  0.23278810424574736
point =  7.51647897875328
probability of observing that point if it came from cluster 0 =  1.3130955479844487e-194
probability of observing that point if it came from cluster 1 =  0.40478903153214324
point =  4.4015884453799305
probability of observing that point if it came from cluster 0 =  1.7608552326706572
probability of observing that point if it came from cluster 1 =  4.369276409513669e-05
point =  4.829953004896432
probability of observing that point if it came from cluster 0 =  0.042920389160167445
probability of observing that point if it came from cluster 1 =  0.0003509441381539681
point =  7.38899752136

The estimated probabilities are far from the truth.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.53929956814716,  mean_2 = 7.926341481180217
var_1 = 0.09702629930617361,  var_2 = 1.0885939043715056


The values are close to the original ones except var_1.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
from scipy.stats import norm

# Updated probabilities
prob_s0_x_updated = [] # P(S_0 | X_i)
prob_s1_x_updated = [] # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_updated = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x_updated.append( pdf_i[0] * prob_c[0] / prob_x_updated )
    prob_s1_x_updated.append( pdf_i[1] * prob_c[1] / prob_x_updated )

# Compare the updated probabilities with the original ones
for i in range(len(data)):
    print("Data point: ", data[i])
    print("Original P(S_0 | X_i): ", prob_s0_x[i])
    print("Updated P(S_0 | X_i): ", prob_s0_x_updated[i])
    print("Original P(S_1 | X_i): ", prob_s1_x[i])
    print("Updated P(S_1 | X_i): ", prob_s1_x_updated[i])
    print()

Data point:  9.594807152750198
Original P(S_0 | X_i):  0.0
Updated P(S_0 | X_i):  0.0
Original P(S_1 | X_i):  1.0
Updated P(S_1 | X_i):  1.0

Data point:  6.99753873643184
Original P(S_0 | X_i):  1.90457023238877e-132
Updated P(S_0 | X_i):  4.201553300654657e-139
Original P(S_1 | X_i):  1.0
Updated P(S_1 | X_i):  1.0

Data point:  7.51647897875328
Original P(S_0 | X_i):  2.1626006729387368e-194
Updated P(S_0 | X_i):  2.7187351605658676e-204
Original P(S_1 | X_i):  1.0
Updated P(S_1 | X_i):  1.0

Data point:  4.4015884453799305
Original P(S_0 | X_i):  0.9999627813203231
Updated P(S_0 | X_i):  0.997971319481113
Original P(S_1 | X_i):  3.7218679676931824e-05
Updated P(S_1 | X_i):  0.002028680518887016

Data point:  4.829953004896432
Original P(S_0 | X_i):  0.9878836613468026
Updated P(S_0 | X_i):  0.8208353447753641
Original P(S_1 | X_i):  0.012116338653197367
Updated P(S_1 | X_i):  0.17916465522463587

Data point:  7.388997521364456
Original P(S_0 | X_i):  4.2103449129381164e-178
Updated

The updated `P(S_j | X_i)` are far from the original ones.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
hard_assignment = []
for i in range(len(data)):
    if prob_s0_x_updated[i] > prob_s1_x_updated[i]:
        hard_assignment.append(0)
    else:
        hard_assignment.append(1)
print(hard_assignment)

[1, 1, 1, 0, 0, 1, 1, 1, 0, 1]
